problem statement : presence of heart disease in the patient
Dataset :-
https://www.kaggle.com/ronitf/heart-disease-uci

In [5]:
import pandas as pd
import numpy as np

In [6]:
# read the dataset
df_heart = pd.read_csv(r'D:\KUNTAL\Python\Data Science Course\Live Classes on Machine Learning\PROJECT\Heart Disease Prediction\heart.csv')
df_heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
# check missing value / NULL
df_heart.isnull().any()

age         False
sex         False
cp          False
trestbps    False
chol        False
fbs         False
restecg     False
thalach     False
exang       False
oldpeak     False
slope       False
ca          False
thal        False
target      False
dtype: bool

In [31]:
df_heart.shape

(303, 14)

In [8]:
# get the target variable
y = df_heart['target']

# get ihe independent variables
X = df_heart.drop(['target'], axis=1)

print(y)
print(X)

0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64
     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       1

In [9]:
# split data into train and test
from sklearn.model_selection import train_test_split

# train:test ratio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [10]:
# Build Random Forest model with Hyperparameter Tuning

# number of trees in Random Forest
n_estimators = [int(x) for x in np.linspace(start=10, stop=80, num=10)]
# number of features to consider at every split
max_features = ['auto','sqrt']
# maximum number of levels at every tree
max_depth = [2,4]
# minimum number of samples required to split a node
min_samples_split = [2,5]
# minimum number of samples required at each leaf node
min_samples_leaf = [1,2]
# method of selecting samples for training each tree
bootstrap = [True, False]


In [11]:
# create the param grid
param_grid = {'n_estimators' : n_estimators,
              'max_features' : max_features,
              'max_depth' : max_depth,
              'min_samples_split' : min_samples_split,
              'min_samples_leaf' : min_samples_leaf,
              'bootstrap' : bootstrap
              }

print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [12]:
# random forest
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

In [13]:
# check GridSearchCV
from sklearn.model_selection import GridSearchCV

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
rf_grid = GridSearchCV(estimator = rf_model, param_grid = param_grid, n_jobs = 4, cv = 10, verbose = 2)

In [18]:
# fit X-train & y_train with rf_grid
rf_grid.fit(X_train, y_train)

Fitting 10 folds for each of 320 candidates, totalling 3200 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  92 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 576 tasks      | elapsed:   10.1s
[Parallel(n_jobs=4)]: Done 1388 tasks      | elapsed:   31.1s
[Parallel(n_jobs=4)]: Done 2520 tasks      | elapsed:   56.5s
[Parallel(n_jobs=4)]: Done 3200 out of 3200 | elapsed:  1.2min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

In [19]:
# the abobe output shows the timing for GridSearchCV as 1.2 min with the below best parameters for the serach
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 4,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 80}

In [20]:
# let's check RandomisedSearchCV
from sklearn.model_selection import RandomizedSearchCV

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html
rf_random = RandomizedSearchCV(estimator = rf_model, param_distributions = param_grid, n_jobs = 4, cv = 10, verbose = 2)

In [22]:
# fit X-train & y_train with rf_random
rf_random.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  86 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.7s finished


RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
              

In [23]:
# the above output shows the RandomisedSearchCV takes only 1.7s to finish
rf_random.best_params_

{'n_estimators': 80,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 4,
 'bootstrap': False}

In [24]:
# compare the best parameters of 2 search
print(rf_grid.best_params_)
print(rf_random.best_params_)

{'bootstrap': True, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 80}
{'n_estimators': 80, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 4, 'bootstrap': False}


In [29]:
# Check the accuracy of GridSearchCV
print(f'Train Accuracy of GridSearchCV : {rf_grid.score(X_train, y_train):.3f}')
print(f'Test Accuracy of GridSearchCV : {rf_grid.score(X_test, y_test):.3f}')

Train Accuracy of GridSearchCV : 0.901
Test Accuracy of GridSearchCV : 0.852


In [30]:
# Check the accuracy of RandomisedSearchCV
print(f'Train Accuracy of RandomisedSearchCV : {rf_random.score(X_train, y_train):.3f}')
print(f'Test Accuracy of RandomisedSearchCV : {rf_random.score(X_test, y_test):.3f}')

Train Accuracy of RandomisedSearchCV : 0.909
Test Accuracy of RandomisedSearchCV : 0.820


In [32]:
# comparison with GridSerachCV and RadomisedSearchCV
# --------------------------------------------------
# here GridSerachCV is taking 1.2 min with totalling 3200 fits
# here RadomisedSearchCV is taking 1.7s with totalling 100 fits
# so for small amount of dataset, you can go for GridSearchCV
# for large amout of dataset, you can go for RadomisedSearchCV